In [1]:
import serial
import time
import pyvisa 
import numpy as np
from IPython.display import Image, display

# You will have to change this to whatever COM port the pico is assigned when
# you plug it in.
# On Windows you can open device manager and look at the 'Ports (COM & LPT)' dropdown
# the pico will show up as 'USB Serial Device'
PICO_PORT = 'COM3'

MHZ = 1000000

conn = None
conn = serial.Serial(PICO_PORT, baudrate = 152000, timeout = 0.1)


## Helpers

In [2]:
# conn.write(b'program\n')

In [3]:
class KeysightScope:
    """
    Helper class using pyvisa for a USB connected Keysight Oscilloscope 
    """
    def __init__(self, 
                 addr='USB?*::INSTR',
                 timeout=1, 
                 termination='\n'
                 ):
        rm = pyvisa.ResourceManager()
        devs = rm.list_resources(addr)
        assert len(devs), "pyvisa didn't find any connected devices matching " + addr
        self.dev = rm.open_resource(devs[0])
        self.dev.timeout = 15_000 * timeout
        self.dev.read_termination = termination
        self.idn = self.dev.query('*IDN?')
        self.read = self.dev.read
        self.write = self.dev.write
        self.query = self.dev.query

    def _get_screenshot(self, verbose=False):
        if verbose == True:
            print('Acquiring screen image...')
        return(self.dev.query_binary_values(':DISPlay:DATA? PNG, COLor', datatype='s'))

    def save_screenshot(self, filepath, verbose=False, inksaver=False):
        if verbose == True:
            print('Saving screen image...')
        result = self._get_screenshot()

        # Keysight scope defaults to inksaving for image save
        if inksaver == True:
            self.dev.write(':HARDcopy:INKSaver 1')
        else: 
            self.dev.write(':HARDcopy:INKSaver OFF')

        with open(f'{filepath}', 'wb+') as ofile:
            ofile.write(bytes(result))

    def set_time_delay(self, time):
        self.write(f':TIMebase:DELay {time}')

    def set_time_scale(self, time):
        self.write(f':TIMebase:SCALe {time}')

my_instrument = KeysightScope()

In [4]:
def insert_string(original_string, string_to_insert, index):

    return original_string[:index] + string_to_insert + original_string[index:]


In [5]:
class Binary_Routines():
    """
    Class handler for `setb` command to send binary tables to the dds-sweeper
    """

    def __init__(self):
        self.mirror_channels = False
        self.debug_run = True
        self.MAX_SIZE = 256

    def catch_response(self):
        resp = conn.readline().decode().strip()
        return resp

    def assert_OK(self):
        resp = conn.readline().decode().strip()
        assert resp == 'ok', 'Expected "ok", received "%s"' % resp

    def debug_on(self):
        conn.write(b'debug on\n')
        self.assert_OK()

    def debug_off(self):
        conn.write(b'debug off\n')
        self.assert_OK()

    def set_mode(self, sweep_mode: int, timing_mode: int, num_channels: int):
        conn.write(b'reset\n')
        self.assert_OK()
        conn.write(b'mode %d %d \n' % (sweep_mode, timing_mode))
        self.assert_OK()
        conn.write(b'setchannels %d\n' % num_channels)
        self.assert_OK()
        if self.debug_run == True:
            print('sending commands to %d channels' % num_channels)

    def allocate(self, start_address: int, instructions: np.ndarray):
        conn.write(b'setb %d %d\n' % (start_address, len(instructions)))
        response = conn.readline().decode()
        if not response.startswith('ready'):
            response += ''.join([r.decode() for r in conn.readlines()])
            raise Exception(f'setb command failed, response: {repr(response)}')
        if self.debug_run == True:
            print(f'Got response: {response}')

    def table_to_memory(self, instructions: np.ndarray):
        conn.write(instructions.tobytes())
        self.assert_OK(), 'table not written correctly'
        if self.debug_run == True:
            print('table written to memory')

    def end_table(self, instructions: np.ndarray):
        conn.write(b'set 4 %d\n' % len(instructions))
        self.assert_OK(), 'table not stopped correctly'
        if self.debug_run == True:
            print('table end command sent')

    def start_routine(self):
        conn.write(b'start\n')
        self.assert_OK(), 'table did not start'
        if self.debug_run == True:
            print('table executed')

    def read_table(self, indices = None):
        conn.write(b'readtable\n')
        for _ in range(self.MAX_SIZE):
            resp = self.catch_response()

            ## No readability hint or end of response
            if indices == None or "End" in resp:
                print(resp)
            else:
                line = resp.split(":")
                str_of_bytes = line[-1]
                for ind in indices:
                    str_of_bytes = insert_string(str_of_bytes, "|", ind * 2)
                line[-1] = str_of_bytes
                print(''.join(line))

            ## termination conditions
            if "End of" in resp:
                break
            elif "Cannot" in resp:
                break
            elif len(resp) == 0:
                break
        # self.assert_OK, 'something went wrong reading table'

    def get_memory_layout(self):
        conn.write(b'getmode\n')
        # self.assert_OK, 
        # self.assert_OK, 'something went wrong getting memory layout'
        mode = self.catch_response()
        channels = self.catch_response()
        memory_layout = self.catch_response()
        print(f"mode {mode}")
        print(channels)
        print(memory_layout)
        # self.assert_OK, 'something went wrong getting memory layout'
        
Binary_Routine_Test = Binary_Routines()

In [6]:
## slightly different logic than the ad9959.c funcs due to python vs c
def get_ftw(freq_out: float, freq_sys: float = 500 * MHZ):

    ftw = (freq_out / freq_sys) * 2**32
    
    return(int(ftw))

def get_pow(phase: float):
    
    pow = round(phase / 360 * (2**14 - 1))

    return pow

def get_asf(amp: float):
    
    asf = round(amp * 1024)

    return asf

## Single Stepping Mode

Single Stepping (mode 0) with timing: `<frequency:int 32> <amplitude:int 16> <phase:int 16> <time: int 32>`. Total of 12 bytes per channel per instruction.

In [7]:
dt = np.dtype([('frequency', np.uint32), 
               ('amplitude', np.uint16), 
               ('phase', np.uint16), 
               ('time', np.uint32)])

f1 = get_ftw(freq_out = 100 * MHZ)
f2 = get_ftw(freq_out = 120 * MHZ)
f3 = get_ftw(freq_out = 110 * MHZ)

a1 = 1023
a2 = 500

p1 = get_pow(phase = 90)
p2 = get_pow(phase = 180)
p3 = get_pow(phase = 270)

t = 2

step_instructions = np.array([
                                (f2, a1, p1, t),
                                (f3, a2, p1, t),
                                (f1, a1, p1, t),
                                (f3, a1, p1, t),
                                 ]
                                 , dtype = dt)

In [8]:
single_step_test = Binary_Routines()

In [9]:
single_step_test.set_mode(sweep_mode=0, timing_mode=1, num_channels=1)
single_step_test.allocate(start_address=0, instructions=step_instructions)
single_step_test.table_to_memory(instructions=step_instructions)
single_step_test.end_table(instructions=step_instructions)

single_step_test.start_routine()

sending commands to 1 channels
Got response: ready for 48 bytes

table written to memory
table end command sent
table executed


In [10]:
single_step_test.read_table(indices=[0, 4, 9, 12])

Instruction Table Dump||||
Offset is 0 | Raw bytes for instruction 0| 7F 00 |F2 04 3D |70 A3| D7 05 10 00 06 00 13 FF
Offset is 15 | Raw bytes for instruction 1| 01 00 |F2 04 38 |51 EB| 85 05 10 00 06 00 11 F4
Offset is 30 | Raw bytes for instruction 2| 05 00 |F2 04 33 |33 33| 33 05 10 00 06 00 13 FF
Offset is 45 | Raw bytes for instruction 3| 01 00 |F2 04 38 |51 EB| 85 05 10 00 06 00 13 FF
End of Instruction Table


In [11]:
single_step_test.get_memory_layout()

mode Mode 0 - Single Steps
Active channels: 1
datatypes = np.dtype([('frequency', np.uint32), ('amplitude', np.uint16), ('phase', np.uint16), ('time', np.uint32)])


In [12]:
single_step_test.catch_response()

''

## Frequency Sweeps

Frequency Sweeps (mode 2) with timing: `<start frequency:int 32> <stop frequency:int 32> <delta:int 32> <rate:int 8> <time:int 32>`. Total of 17 bytes per channel per instruction.

In [13]:
dt = np.dtype([('start frequency', np.uint32), 
               ('stop frequency', np.uint32), 
               ('delta', np.uint32), 
               ('rate', np.uint8), 
               ('time', np.uint32)])

f1 = get_ftw(freq_out = 100 * MHZ)
f2 = get_ftw(freq_out = 120 * MHZ)
f3 = get_ftw(freq_out = 110 * MHZ)

d = 100
r = 1
t = 2

freq_sweep_table = np.array([
                            (f1, f3, d, r, t),
                            (f3, f2, d, r, t),
                            (f2, f1, d, r, t),
                            (f1, f1, d, r, t),
                            (f2, f2, d, r, t),
                            (f3, f3, d, r, t),
                            (f3, f1, d, r, t),
                            (f1, f2, d, r, t),
                            (f2, f3, d, r, t),
                            (f1, f1, d, r, t),
                                 ]
                                 , dtype = dt)

In [14]:
freq_sweep_test = Binary_Routines()
freq_sweep_test.set_mode(sweep_mode=2, timing_mode=1, num_channels=1)
freq_sweep_test.allocate(start_address=0, instructions=freq_sweep_table)
freq_sweep_test.table_to_memory(instructions=freq_sweep_table)
freq_sweep_test.end_table(instructions=freq_sweep_table)
freq_sweep_test.start_routine()

sending commands to 1 channels
Got response: ready for 170 bytes

table written to memory
table end command sent
table executed


In [15]:
freq_sweep_test.read_table(indices=[0, 7, 13, 17])

Instruction Table Dump||||
Offset is 0 | Raw bytes for instruction 0| FF 00 F2 04 |33 33 33 33| 07 01 |01 08 00 00 00 64 09 00 00 00 00 0A 38 51 EB 85 03 80 43 10
Offset is 30 | Raw bytes for instruction 1| FF 00 F2 04 |38 51 EB 85| 07 01 |01 08 00 00 00 64 09 00 00 00 00 0A 3D 70 A3 D7 03 80 43 10
Offset is 60 | Raw bytes for instruction 2| 0F 00 F2 04 |33 33 33 33| 07 01 |01 08 FF FF FF FF 09 00 00 00 64 0A 3D 70 A3 D7 03 80 43 10
Offset is 90 | Raw bytes for instruction 3| 0F 00 F2 04 |33 33 33 33| 07 01 |01 08 00 00 00 64 09 00 00 00 00 0A 33 33 33 33 03 80 43 10
Offset is 120 | Raw bytes for instruction 4| 0F 00 F2 04 |3D 70 A3 D7| 07 01 |01 08 00 00 00 64 09 00 00 00 00 0A 3D 70 A3 D7 03 80 43 10
Offset is 150 | Raw bytes for instruction 5| 0F 00 F2 04 |38 51 EB 85| 07 01 |01 08 00 00 00 64 09 00 00 00 00 0A 38 51 EB 85 03 80 43 10
Offset is 180 | Raw bytes for instruction 6| 0F 00 F2 04 |33 33 33 33| 07 01 |01 08 FF FF FF FF 09 00 00 00 64 0A 38 51 EB 85 03 80 43 10
Offset is 21

In [16]:
freq_sweep_test.get_memory_layout()

mode Mode 2 - Frequency Sweeps
Active channels: 1
datatypes = np.dtype([('start frequency', np.uint32), ('stop frequency', np.uint32), ('delta', np.uint32), ('rate', np.uint8),


## Phase Sweeps

Phase Sweeps (mode 3) with timing: `<start phase:int 16> <stop phase:int 16> <delta:int 16> <rate:int 8> <time:int 32>`. Total of 11 bytes per channel per instruction.

In [17]:
for i in range(4):
    conn.write(b'setfreq %d 100000000\n' % i)
    Binary_Routine_Test.catch_response()

    conn.write(b'setphase %d 0\n' % i)
    Binary_Routine_Test.catch_response()

    conn.write(b'setamp %d 1023\n' % i)
    Binary_Routine_Test.catch_response()

dt = np.dtype([('start phase0', np.uint16), 
               ('stop phase0', np.uint16), 
               ('delta0', np.uint16), 
               ('rate0', np.uint8),
               ('time0', np.uint32),
               ('start phase1', np.uint16), 
               ('stop phase1', np.uint16), 
               ('delta1', np.uint16), 
               ('rate1', np.uint8), 
               ('time1', np.uint32)])

p0 = get_pow(phase = 0)
p1 = get_pow(phase = 90)
p2 = get_pow(phase = 180)
p3 = get_pow(phase = 270)
p4 = get_pow(phase = 360)

d = 1
r = 1
t = 2


phase_sweep_instructions = np.array([
                                    (0, 0, d, r, t, p0, p2, d, r, t),
                                    (0, 0, d, r, t, p2, p1, d, r, t),
                                    (0, 0, d, r, t, p1, p0, d, r, t),
                                    # (0, 0, d, r, t, p0, p1, d, r, t),
                                    # (0, 0, d, r, t, p1, p2, d, r, t),
                                    # (0, 0, d, r, t, p2, p0, d, r, t),
                                 ]
                                 , dtype = dt)

In [18]:
Binary_Routine_Test.catch_response()

''

In [19]:
phase_sweep_test = Binary_Routines()
phase_sweep_test.debug_off()
phase_sweep_test.set_mode(sweep_mode=3, timing_mode=1, num_channels=2)

phase_sweep_test.allocate(start_address=0, instructions=phase_sweep_instructions)
phase_sweep_test.table_to_memory(instructions=phase_sweep_instructions)

phase_sweep_test.end_table(instructions=phase_sweep_instructions)
phase_sweep_test.start_routine()

sending commands to 2 channels
Got response: ready for 66 bytes

table written to memory
table end command sent
table executed


In [20]:
phase_sweep_test.get_memory_layout()

mode Mode 3 - Phase Sweeps
Active channels: 2
datatypes = np.dtype([('start phase', np.uint16), ('stop phase', np.uint16), ('delta', np.uint16), ('rate', np.uint8), ('time',


In [21]:
phase_sweep_test.read_table()

Instruction Table Dump:
Offset is: 0 | Raw bytes for instruction 0: 7F 00 12 05 00 00 07 01 01 08 00 04 00 00 09 00 00 00 00 0A 00 00 00 00 03 C0 43 10 00 22 05 00 00 07 01 01 08 00 04 00 00 09 00 00 00 00 0A 80 00 00 00 03 C0 43 10
Offset is: 55 | Raw bytes for instruction 1: 1F 00 12 05 00 00 07 01 01 08 00 04 00 00 09 00 00 00 00 0A 00 00 00 00 03 C0 43 10 00 22 05 10 00 07 01 01 08 FF FF FF FF 09 00 04 00 00 0A 80 00 00 00 03 C0 43 10
Offset is: 110 | Raw bytes for instruction 2: 0C 00 12 05 00 00 07 01 01 08 00 04 00 00 09 00 00 00 00 0A 00 00 00 00 03 C0 43 10 00 22 05 00 00 07 01 01 08 FF FF FF FF 09 00 04 00 00 0A 40 00 00 00 03 C0 43 10
End of Instruction Table


In [22]:
print(p0, hex(p0))
print(p1, hex(p1))
print(p2, hex(p2))


0 0x0
4096 0x1000
8192 0x2000


In [ ]:
# #### not always necessary, just if you want to move back to tests.ipynb
# conn.close()

In [24]:
# conn.write(b'program\n')

In [25]:
# for i in range(4):
#     conn.write(b'setfreq %d 100000000\n' % i)
#     Binary_Routine_Test.catch_response()

#     conn.write(b'setphase %d 0\n' % i)
#     Binary_Routine_Test.catch_response()

#     conn.write(b'setamp %d 1023\n' % i)
#     Binary_Routine_Test.catch_response()

# dt = np.dtype([('start amp0', np.uint16), 
#                ('stop amp0', np.uint16), 
#                ('delta0', np.uint16), 
#                ('rate0', np.uint8),
#                ('time0', np.uint32),
#                ('start amp1', np.uint16), 
#                ('stop amp1', np.uint16), 
#                ('delta1', np.uint16), 
#                ('rate1', np.uint8), 
#                ('time1', np.uint32)])

# p0 = get_pow(phase = 0)
# p1 = get_pow(phase = 90)
# p2 = get_pow(phase = 180)
# p3 = get_pow(phase = 270)
# p4 = get_pow(phase = 360)

# d = 1
# r = 1
# t = 2


# phase_sweep_instructions = np.array([
#                                     (0, 0, d, r, t, p0, p2, d, r, t),
#                                     (0, 0, d, r, t, p2, p1, d, r, t),
#                                     (0, 0, d, r, t, p1, p0, d, r, t),
#                                     (0, 0, d, r, t, p0, p1, d, r, t),
#                                     (0, 0, d, r, t, p1, p2, d, r, t),
#                                     (0, 0, d, r, t, p2, p0, d, r, t),
#                                  ]
#                                  , dtype = dt)